In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
from pymongo import MongoClient
from bson.objectid import ObjectId

import time,datetime
import os

import pandas as pd
import pandas_datareader.data as web
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style

from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [4]:
conn = MongoClient()
db = conn.Tracker
collection = [db.james]# db.db2, db.dn2, db.james, db.leo
clean_data = []
for col in collection:
    cursor = col.find({})
    df = pd.DataFrame(list(cursor))

In [5]:
df.replace('', np.nan, inplace=True)

In [6]:
df.fillna(method='ffill', inplace=True)
df.isnull().sum().sum()

0

In [7]:
df['hr_value'] = df['hr_value'].astype(float)
df['o2_value'] = df['o2_value'].astype(float)

In [8]:
dftspdiff = df['timestamp'].diff()
dftfdiffSTD = dftspdiff.rolling(20).std()

# find the place that std is smaller than 5
lis = []
changeFlag = 0
tempt = [0]
for i in range(len(dftfdiffSTD)):
    if dftfdiffSTD[i] < 5 and changeFlag == True and len(tempt) < 2:
        tempt.append(i)
        changeFlag = False
    if dftfdiffSTD[i] >= 5 and changeFlag == False and len(tempt) < 2:
        tempt.append(i-1)
        changeFlag = True
    if len(tempt) == 2:
        lis.append(tempt)
        tempt = []
print(lis)

[[0, 108], [212, 238], [259, 371], [407, 511], [532, 650], [672, 782], [805, 830], [1072, 1176], [1197, 1485], [1506, 1757], [1845, 1867], [1888, 2240], [2261, 2263], [2313, 2381], [2466, 2859], [2994, 3101], [3129, 3864], [3980, 4169], [4275, 4373], [4448, 4797], [4818, 4986], [5018, 5024], [5115, 5533], [5554, 5886], [5912, 5926], [6046, 6065], [6087, 6151], [6265, 6295], [6360, 6420], [6554, 6582], [6619, 6624], [6683, 6748], [6769, 6820], [6886, 6948], [6970, 7108], [7129, 7137], [7169, 7193], [7373, 7507], [7554, 7671], [7714, 7738], [7761, 7852], [7874, 7905], [7979, 8428], [8576, 8598], [8680, 8735], [8758, 8905], [8929, 8968], [9013, 9135], [9163, 9207], [9309, 9349], [9386, 9454], [9513, 9747], [9782, 10038], [10059, 10346], [10504, 10518], [10523, 10532], [10539, 10541], [10577, 10656], [10711, 10808], [10856, 10966], [11005, 11276], [11297, 11371], [11418, 11426], [11459, 11700], [11763, 11778], [11799, 14477], [14498, 14706], [14738, 15224], [15245, 15379], [15443, 15465], 

In [12]:
data_with_gap = []
for i in lis:
    data_with_gap.append(df[i[0]:i[1]])

In [17]:
data_with_gap[0].head()

,_id,day,hour,hr_value,latitude,longitude,minute,month,o2_value,second,step_value,timestamp,year
0,5c9b3935c4e5a668b7e9be2c,27,16,84.0,24.94519427,121.37227724,49,3,95.0,47,0,1.553677e+09,2019
1,5c9b393fc4e5a668b7e9be2e,27,16,84.0,24.94519427,121.37227724,49,3,95.0,57,0,1.553677e+09,2019
2,5c9b3949c4e5a668b7e9be30,27,16,84.0,24.94519427,121.37227724,50,3,95.0,7,0,1.553677e+09,2019
3,5c9b3953c4e5a668b7e9be32,27,16,84.0,24.94519427,121.37227724,50,3,95.0,17,0,1.553677e+09,2019
4,5c9b395dc4e5a668b7e9be34,27,16,79.0,24.94519427,121.37227724,50,3,98.0,27,0,1.553677e+09,2019
